# 1. Low level features

In [ ]:
import numpy as np
import pandas as pd
import pickle
import importlib

import matplotlib.pyplot as plt
import plotly.graph_objects as go

import util

In [ ]:
# Heart rate time history image with color label for each hour
file_full = [f'a{i:02d}' for i in range(1, 21)] + \
    [f'b{i:02d}' for i in range(1, 6)] + \
    [f'c{i:02d}' for i in range(1, 11)]

for file in file_full:
    print(file)
    with open('data/raw/' + file + '.pkl', 'rb') as f:
        res = pickle.load(f)

    total_hour = int(len(res['apn']) / 60) + 1
    for hour in range(total_hour):
        minute_start = hour * 60
        if hour == total_hour - 1:
            minute_end = len(res['apn'])
        else:
            minute_end = (hour + 1) * 60
            
        fig = util.plot_hr_apn(res, minute_start, minute_end)
        fig.write_image('image_HR/' + f'{file}_minute{minute_start}.png',scale=3)

In [ ]:
# Heart rate data for each patient
file_full = [f'a{i:02d}' for i in range(1, 21)] + \
    [f'b{i:02d}' for i in range(1, 6)] + \
    [f'c{i:02d}' for i in range(1, 11)] + \
    [f'x{i:02d}' for i in range(1, 36)]

for file in file_full:
    print(file)
    with open('data/processed/' + file + '.pkl', 'rb') as f:
        data = pickle.load(f)
    
    t_hr, hr = [], []
    for minute in range(len(data['apn'])):
        t_hr_, hr_ = util.get_heart_rate(data['ecg'][minute])
        t_hr_ = t_hr_ / 60 + minute # Unit in minutes
        t_hr.append(t_hr_)
        hr.append(hr_)    
        
    hr = np.hstack(hr)
    t_hr = np.hstack(t_hr)
    res = {'hr': hr, 't': t_hr}
    with open('features/HR_' + file + '.pkl', 'wb') as f:
        pickle.dump(res, f)

# 2. High level features for model fitting

In [6]:
from hrvanalysis import get_time_domain_features, get_csi_cvi_features
import numpy as np
import pandas as pd
import pickle
from scipy import signal

import util

In [9]:
def extract_features(file_names):
    df = pd.DataFrame()
    b, a = signal.butter(3, 0.1)
    fs_new = 2.4 # optimized from hyper-parameter tuning

    for file in file_names:
        print(file)
        
        # Load files
        with open('data/processed/' + file + '.pkl', 'rb') as f:
            res = pickle.load(f)
            apn = res['apn']
            group = util.ecg_diagnose(apn) if file[0] == 'x' else file[0].upper() 

        with open('features/HR_' + file + '.pkl', 'rb') as f:
            res = pickle.load(f)
            hr = res['hr']
            t_hr = res['t'] # in minute
            
        # Remove outliers 
        idx_valid = (hr < 2) & (hr > 0.5)
        hr, t_hr = hr[idx_valid], t_hr[idx_valid]
        
        # Filter out high-frequency noise
        hr_smth = signal.filtfilt(b, a, hr)
        
        # Resample data for frequency-domain analysis
        t_interp = np.arange(t_hr[0], t_hr[-1], 1 / fs_new / 60)
        hr_interp = np.interp(t_interp, t_hr, hr_smth)
        
        # Extract features from each segment
        for minute in range(len(apn) - 4):
            fea_dict = {}
            idx_1min = (t_hr > minute) & (t_hr < minute + 1)
            idx_5min = (t_hr > minute) & (t_hr < minute + 5)
            data_1min, data_5min = hr_smth[idx_1min], hr_smth[idx_5min]
            
            # Discard segment if less than 30 heart beats detected
            if len(data_1min) < 30: 
                continue
                
            # Time-domain features for data_1min
            md = np.median(data_1min)
            fea_dict.update({
                'md_1min': md,
                'min_r_1min': data_1min.min() - md,
                'max_r_1min': data_1min.max() - md,
                'p25_r_1min': np.percentile(data_1min, 0.25) - md,
                'p75_r_1min': np.percentile(data_1min, 0.75) - md,
                'mean_r_1min': data_1min.mean() - md,
                'std_1min': data_1min.std(),
                'acf1_1min': pd.Series(data_1min).autocorr(1),
                'acf2_1min': pd.Series(data_1min).autocorr(2),
            })
            
            # Time-domain features for data_5min
            md = np.median(data_5min)
            fea_dict.update({
                'md_5min': md,
                'min_r_5min': data_5min.min() - md,
                'max_r_5min': data_5min.max() - md,
                'p25_r_5min': np.percentile(data_5min, 0.25) - md,
                'p75_r_5min': np.percentile(data_5min, 0.75) - md,
                'mean_r_5min': data_5min.mean() - md,
                'std_5min': data_5min.std(),
                'acf1_5min': pd.Series(data_5min).autocorr(1),
                'acf2_5min': pd.Series(data_5min).autocorr(2),
            })
            
            # Heart rate variability
            nn_intervals = (np.diff(t_hr[idx_1min]) * 1000 * 60).astype(int) # Unit in ms
            time_domain_features = get_time_domain_features(nn_intervals)
            nonlinear_features = get_csi_cvi_features(nn_intervals)
            fea_dict.update(time_domain_features)
            fea_dict.update(nonlinear_features)
            
            # Frequency-domain features
            freqs, psd = signal.welch(
                x=hr_interp[(t_interp > minute) & (t_interp < minute + 5)], 
                fs=fs_new)
            fea_dict.update({
                'peak': psd.max(),
                'f_peak': freqs[np.argmax(psd)],
                'area_total': psd.sum(),
                'area_lf': psd[freqs < 1e-2].sum(),
                'area_hf': psd[freqs > 1e-2].sum(),
                'area_ratio': psd[freqs > 1e-2].sum() / psd[freqs < 1e-2].sum(),
            })
            
            # Label information
            fea_dict.update({
                'apn': apn[minute + 2],
                'group': group,
                'file': file,
            })
            df = df.append(fea_dict, ignore_index=True)
                    
    df['apn'] = df['apn'].astype(int)
    return df

In [10]:
train_df = pd.read_csv('resources\File_train.csv')
df = extract_features(train_df['file'])
df.dropna(inplace=True)
df.to_csv('features/feature.csv', index=False)

x32
a13
c09
x29
b02
c03
x16
a08
a15
x34
a14
a12
a07
c05
a04
a06
x28
a09
b01
x33
c10
c04
a16
x20
a10
a19
x27
x01
x07
x19
a17
b05
x18
b04
x11
x17
a05
x22
x35
x31
x13
a01
c02
x24
x21
x02
x08
x09
x14
b03
c08
a11
a02
x30
x12
c07


OSError: [Errno 22] Invalid argument: 'features\x0ceature.csv'